In [ ]:
import sys
path2cpp_pkg = "/Users/mariusmahiout/Documents/repos/ising_core/build"
sys.path.append(path2cpp_pkg)
import ising

import os
os.chdir("/Users/mariusmahiout/Documents/repos/ising_core/python/src")
import utils as utils
import model_eval as eval
import isingfitter as fitter
import misc_plotting as plotting

os.chdir("..")

import numpy as np
import plotly
from IPython.display import display, HTML

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

In [ ]:
num_units = 50
num_sims = 15_000
num_burn = 5000

In [ ]:
##############
# SIMULATION #
##############

# setting up model
beta = 1.3
h = np.random.uniform(-.3 * beta, .3 * beta, num_units)
J = np.random.normal(0,  beta / np.sqrt(num_units), (num_units, num_units))
for i in range(num_units):
    J[i, i] = 0
    for j in range(i+1, num_units):
        J[j, i] = J[i, j]

true_model = ising.EqModel(J, h)

# simulating
true_sim = true_model.simulate(num_sims, num_burn)

In [ ]:
#train_sample, test_sample = utils.get_train_test_samples(true_sim)
train_sample = true_sim
test_sample = true_sim

# Gradient ascent

In [ ]:
num_sims = 50_000
num_burn = 2500
lr = 0.1
win_size = 10
tol_ml = 1e-4
tol_pl = 1e-6
max_steps = 1

In [ ]:
##############
# LIKELIHOOD #
##############

# setting up model
h_init = np.random.uniform(-1.5, 1.5, num_units)
J_init = np.random.normal(0,  1,  (num_units, num_units))
J_init = (J_init.T + J_init) * np.sqrt(2) / 2
np.fill_diagonal(J_init, 0)

ml_model = ising.EqModel(J_init, h_init)

ml_fitter = fitter.EqFitter(ml_model)
ml_fitter.TAP(train_sample)

# inference
use_llh = False

ml_fitter.maximize_likelihood(
    sample=train_sample,
    max_steps=1000, 
    learning_rate=0.1,
    win_size = win_size,
    tolerance= tol_ml, 
    num_sims=num_sims, 
    num_burn=num_burn,
    calc_llh=use_llh
)

# ml_fitter.maximize_likelihood(
#     sample=train_sample,
#     max_steps=1000, 
#     learning_rate=0.05,
#     win_size = win_size,
#     tolerance= tol_ml, 
#     num_sims=num_sims, 
#     num_burn=num_burn,
#     calc_llh=use_llh
# )

# ml_fitter.maximize_likelihood(
#     sample=train_sample,
#     max_steps=1500, 
#     learning_rate=0.01,
#     win_size = win_size,
#     tolerance= tol_ml, 
#     num_sims=num_sims, 
#     num_burn=num_burn,
#     calc_llh=use_llh
# )

# ml_fitter.maximize_likelihood(
#     sample=train_sample,
#     max_steps=1500, 
#     learning_rate=0.005,
#     win_size = win_size,
#     tolerance= tol_ml, 
#     num_sims=num_sims, 
#     num_burn=num_burn,
#     calc_llh=use_llh
# )

In [ ]:
plotting.convergence_plot(ml_fitter, plot_llh=use_llh, path=f"./analyses/eq_convergence/ml{num_units}.pdf")

In [ ]:
#####################
# PSEUDO-LIKELIHOOD #
#####################

# setting up model
h_init = np.random.uniform(-1.5, 1.5, num_units)
J_init = np.random.normal(0,  1,  (num_units, num_units))
J_init = (J_init.T + J_init) * np.sqrt(2) / 2
np.fill_diagonal(J_init, 0)

pl_model = ising.EqModel(J_init, h_init)
pl_fitter = fitter.EqFitter(pl_model)
pl_fitter.TAP(train_sample)

# inference
pl_fitter.maximize_likelihood(
    sample=train_sample, 
    max_steps=1000, 
    learning_rate=0.1,
    win_size = win_size,
    tolerance= tol_pl, 
    num_sims=0, 
    num_burn=0,
)

# pl_fitter.maximize_likelihood(
#     sample=train_sample, 
#     max_steps=1000, 
#     learning_rate=0.05,
#     win_size = win_size,
#     tolerance= tol_pl, 
#     num_sims=0, 
#     num_burn=0,
# )

# pl_fitter.maximize_likelihood(
#     sample=train_sample, 
#     max_steps=1500, 
#     learning_rate=0.01,
#     win_size = win_size,
#     tolerance= tol_pl, 
#     num_sims=0, 
#     num_burn=0,
# )

# pl_fitter.maximize_likelihood(
#     sample=train_sample, 
#     max_steps=1500, 
#     learning_rate=0.005,
#     win_size = win_size,
#     tolerance= tol_pl, 
#     num_sims=0, 
#     num_burn=0,
# )

In [ ]:
plotting.convergence_plot(pl_fitter, plot_llh=use_llh, path=f"./analyses/eq_convergence/pl{num_units}.pdf")

In [ ]:
nmf_model = ising.EqModel(J_init, h_init)
nmf_fitter = fitter.EqFitter(nmf_model)
nmf_fitter.naive_mean_field(train_sample)

tap_model = ising.EqModel(J_init, h_init)
tap_fitter = fitter.EqFitter(tap_model)
tap_fitter.TAP(train_sample)

In [ ]:
ml_sim = ml_model.simulate(num_sims, num_burn)
pl_sim = pl_model.simulate(num_sims, num_burn)
nmf_sim = nmf_model.simulate(num_sims, num_burn)
tap_sim = tap_model.simulate(num_sims, num_burn)

In [ ]:


labels = ["nMF", "TAP", "PLLH", "ML"]
metadata = utils.get_metadata(
    num_units=num_units,
    is_empirical_analysis=False,
    eq_inv_methods=labels,
    num_sims=num_sims,
    true_fields="uniform(-.3 * beta, .3 * beta); beta=1.3",
    true_couplings="normal(0,  beta / sqrt(num_units)); symmetric, beta=1.3",
    num_steps=max_steps,
    learning_rate=lr,
    is_converged=None,
    num_sims_ml=num_sims,
    num_burn_ml=num_burn,
)


analysis_name = "eq_test"
bin_width = 0
analysis_path = utils.get_analysis_path(analysis_name, num_units, bin_width)

layout_spec = {
    # ("fields", "scatter"): (1, 1),
    # ("means", "scatter"): (1, 2),
    # ("couplings", "scatter"): (2, 1),
    # ("pcorrs", "scatter"): (2, 2),
    ("num-distr", "histogram"): (1, 1),

}

ising_eval = eval.IsingEval(
    analysis_path=analysis_path,
    metadata=metadata,
    true_model=true_model,
    est_models=[nmf_model, tap_model, pl_model, ml_model],
    true_sample=test_sample,
    est_samples=[nmf_sim, tap_sim, pl_sim, ml_sim],
    labels=labels,
    layout_spec=layout_spec
)
ising_eval.generate_plots(is_pdf=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def get_coactivation_distribution(states: np.ndarray):
    num_units, num_bins = states.shape
    rel_freq = np.zeros(num_units + 1)
    for bin in range(num_bins):
        active_units = states[:, bin] == 1
        num_active = int(np.count_nonzero(active_units))
        rel_freq[num_active] += 1 / num_bins
    return rel_freq


coac_distr = get_coactivation_distribution(test_sample.getStates())
nmf_coac_distr = get_coactivation_distribution(nmf_sim.getStates())
tap_coac_distr = get_coactivation_distribution(tap_sim.getStates())

plt.plot(coac_distr, color="grey")
plt.plot(nmf_coac_distr, color="orange")
plt.plot(tap_coac_distr, color="slateblue")
plt.show()

# Ideas to investigate:

- Relationship between closeness of parameters (est & true) and and closeness of observables (e.g., means and pcorrs)